In [2]:
import numpy as np

In [3]:
#input layer
class inputLayer:

    def __init__(self, number_of_input): 
        self.number_of_input = number_of_input
    
    #to add the vlaues in the network 
    def use(self, input_matrix):
        if isinstance(input_matrix,np.ndarray):
            self.out_matrix = input_matrix
            return input_matrix
        else:
            self.out_matrix = np.array(input_matrix)
            return self.out_matrix
    
    def fetchInputValues(self):
        return self.out_matrix

#hidden and output layers
class layers:
    
    def __init__(self, number_of_input, number_of_neurons, activation='relu'):
        low, high = -1, 1
        self.weight_matrix = np.random.uniform(low, high, size=(number_of_input, number_of_neurons))
        self.bias = np.random.uniform(low, high, size=(1, number_of_neurons))
        self.activation = activation

    def apply_activation(self, values):
        if self.activation == 'relu':
            return np.maximum(0, values)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-values))
        elif self.activation == 'tanh':
            return np.tanh(values)
        elif self.activation == 'softmax':
            e_x = np.exp(values - np.max(values, axis=-1, keepdims=True))
            return e_x / np.sum(e_x, axis=-1, keepdims=True)
        else:
            return values  # No activation (identity)

    def compute_derivative(self, values):
        if self.activation == 'relu':
            return (values > 0).astype(float)
        elif self.activation == 'sigmoid':
            sigmoid_output = self.apply_activation(values)
            return sigmoid_output * (1 - sigmoid_output)
        elif self.activation == 'tanh':
            tanh_output = self.apply_activation(values)
            return 1 - tanh_output**2
        elif self.activation == 'softmax':
            softmax_output = self.apply_activation(values)
            return softmax_output * (1 - softmax_output)  # Note: This derivative might not be used often for softmax
        else:
            return np.ones_like(values)  # Derivative of the identity function is 1

    def forward(self, input_matrix):
        values = np.dot(input_matrix, self.weight_matrix) + self.bias
        return self.apply_activation(values)
    


In [4]:
class layersT:
    
    def __init__(self, number_of_input, number_of_neurons,activation='relu',wig=None,bia=None):
        low, high = -1, 1
        self.weight_matrix = wig
        self.bias = bia
        self.activation = activation

    def apply_activation(self, values):
        if self.activation == 'relu':
            return np.maximum(0, values)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-values))
        elif self.activation == 'tanh':
            return np.tanh(values)
        elif self.activation == 'softmax':
            e_x = np.exp(values - np.max(values, axis=-1, keepdims=True))
            return e_x / np.sum(e_x, axis=-1, keepdims=True)
        else:
            return values  # No activation (identity)

    def compute_derivative(self, values):
        if self.activation == 'relu':
            return (values > 0).astype(float)
        elif self.activation == 'sigmoid':
            sigmoid_output = self.apply_activation(values)
            return sigmoid_output * (1 - sigmoid_output)
        elif self.activation == 'tanh':
            tanh_output = self.apply_activation(values)
            return 1 - tanh_output**2
        elif self.activation == 'softmax':
            softmax_output = self.apply_activation(values)
            return softmax_output * (1 - softmax_output)  # Note: This derivative might not be used often for softmax
        else:
            return np.ones_like(values)  # Derivative of the identity function is 1

    def forward(self, input_matrix):
        values = np.dot(input_matrix, self.weight_matrix) + self.bias
        self.neuronValues =  self.apply_activation(values)
        #print(f'r =  {self.neuronValues} values = {values}') #debugging
        return self.neuronValues

In [5]:
class Model:

    #to intiallize the model
    def __init__(self) -> None:
        
        inputNumber = 3 # input nodes
        self.numberOfLayers = 3 #number of hidden layer and output layer

        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = int(input('number of input: '))
            numberOfNeurons = int(input('number of neurons: '))
            activation = input('Enter the activation funtion: ')
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = self.inLayerObj.use(data)
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.array([np.multiply(pridiction,layers.compute_derivative(pridiction))])
        prevNuronValues = np.array([self.listOfHiddenLayers[i-1].neuronValues])
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        print(layers.bias) #debugging
        i -= 1

        while i:
            
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = np.array([layers.compute_derivative(newValues)])
            cur_gradient = np.array(np.multiply(derValues.T,pre_w))
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = np.array([self.listOfHiddenLayers[i-1].neuronValues])
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            print(layers.bias) #debugging

            i -= 1

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = np.array([layers.neuronValues])
        derValues = np.array(layers.compute_derivative(newValues))
        cur_gradient = np.array(np.dot(pre_w,gradient.T))
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = np.array([data])
        weights = layer.weight_matrix
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        layers.bias = layers.bias - learningRate*(pr)
    

In [6]:
wig1 = np.array([[-0.80377147, -0.63344107],
                 [0.43278832, -0.02537794],
                 [-0.71144554, 0.79602224]])
wig2 = np.array([[ 0.30313998,-0.651056],
                 [ 0.11990958, -0.68028781]])
wig3 = np.array([[-0.13948543, -0.15244189 ,0.02360036],
                 [ 0.15260582 ,-0.41243941 ,0.62087499]])

b1 = np.array([-0.20960194,-0.86772309])
b2 = np.array([-0.04218223, 0.71097538])
b3 = np.array([-0.45757658, -0.6612067, 0.39908153])

testInputs = [3,2,'relu',wig1,b1,2,2,'relu',wig2,b2,2,3,'softmax',wig3,b3]

class ModelTest:

    #to intiallize the model
    def __init__(self) -> None:
        
        inputNumber = 3 # input nodes
        self.numberOfLayers = 3 #number of hidden layer and output layer

        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)
        ind = 0
        for i in range(self.numberOfLayers):
            number_of_input = testInputs[ind]
            ind += 1
            numberOfNeurons = testInputs[ind]
            ind += 1
            activation = testInputs[ind]
            ind += 1
            w = testInputs[ind]
            ind += 1
            b = testInputs[ind]
            ind += 1
            hidden_layer_obj = layersT(number_of_input,numberOfNeurons,activation,w,b)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = self.inLayerObj.use(data)
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)
        
        # print(costFuntion)
        # print(f"prid - {pridiction}")

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.array([np.multiply(pridiction,layers.compute_derivative(pridiction))])
        prevNuronValues = np.array([self.listOfHiddenLayers[i-1].neuronValues])
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        print(layers.bias) #debugging
        i -= 1

        while i:
            
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = np.array([layers.compute_derivative(newValues)])
            cur_gradient = np.array(np.multiply(derValues.T,pre_w))
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = np.array([self.listOfHiddenLayers[i-1].neuronValues])
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights
            gradient = pr
            i -= 1

            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            print(layers.bias) #debugging

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = np.array([layers.neuronValues])
        #print(f'newvalues =  {newValues}') #debugging
        #print(pre_w) #debugging
        derValues = np.array(layers.compute_derivative(newValues))
        #print(f' der = {derValues}') #debugging
        cur_gradient = np.array(np.dot(pre_w,gradient.T))
        #print(f'cur-gra = {cur_gradient}') #debugging
        pr = np.multiply(derValues,cur_gradient.T)
        #print(f'pr = {pr}') #debugging
        prevNuronValues = np.array([data])
        weights = layer.weight_matrix
        #print(weights) #debugging
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        print(f'upweights = {upWeights.T}') #debugging
        layers.bias = layers.bias - learningRate*(pr)
        print(layers.bias) #debugging

In [7]:
nn = ModelTest()

'''
inputs here

input layer = input 3 (change in Model() if req)

hidden layer 1 = input - 3, nurons - 2
hidden layer 2 = input - 2, nurons - 2
hidden layer 2 = input - 2, nurons - 3 (output)
'''

'\ninputs here\n\ninput layer = input 3 (change in Model() if req)\n\nhidden layer 1 = input - 3, nurons - 2\nhidden layer 2 = input - 2, nurons - 2\nhidden layer 2 = input - 2, nurons - 3 (output)\n'

In [8]:
data = np.array([1,2,3])
nn.train(data,0)


[[-0.4486703  -0.66516052  0.3843125 ]]
[[-0.04315217  0.70484002]]
upweights = [[-0.80479621 -0.6286043 ]
 [ 0.43073884 -0.0157044 ]
 [-0.71451976  0.81053254]]
[[-0.04417691  0.70967679]]


In [9]:
nn.view_weights()

weights 
 - [[-0.80377147 -0.63344107]
 [ 0.43278832 -0.02537794]
 [-0.71144554  0.79602224]]
bias - [-0.20960194 -0.86772309]
weights 
 - [[-0.80479621 -0.6286043 ]
 [ 0.43073884 -0.0157044 ]
 [-0.71451976  0.81053254]]
bias - [[-0.04417691  0.70967679]]
weights 
 - [[-0.13896816 -0.15267153  0.02274258]
 [ 0.15387189 -0.41300147  0.6187755 ]]
bias - [[-0.4486703  -0.66516052  0.3843125 ]]


In [10]:
a1 = np.array([[0,0.836146]])
a2 = np.array([[0.00969939, 0.06135357]])

b = np.array( [[ 0.30313998 ,-0.651056  ],
 [ 0.11990958, -0.68028781]])

z = np.dot(a2.T,a1)
print(b.T-(0.1*z)) #debugging
z

[[ 0.30313998  0.11909857]
 [-0.651056   -0.68541786]]


array([[0.        , 0.00811011],
       [0.        , 0.05130054]])